# Data manipuation for gelocation dataset and creating junction table to avoid many to many relationship between seller, customer and geolocation dataset.

In [3]:
import pandas as pd

In [4]:
geo_loc = pd.read_csv('./geolocation.csv')
geo_loc.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [5]:
states = geo_loc['geolocation_state'].unique()

In [6]:
states

array(['SP', 'RN', 'AC', 'RJ', 'ES', 'MG', 'BA', 'SE', 'PE', 'AL', 'PB',
       'CE', 'PI', 'MA', 'PA', 'AP', 'AM', 'RR', 'DF', 'GO', 'RO', 'TO',
       'MT', 'MS', 'RS', 'PR', 'SC'], dtype=object)

In [7]:
u_states = pd.DataFrame({'state_code':states})
u_states.to_csv('./states.csv')

In [9]:
zip_mean_lat_long = geo_loc.groupby(['geolocation_zip_code_prefix']).agg({'geolocation_lat':'mean','geolocation_lng':'mean'}) # grouping on state and zipcode aggregating on lat and long mean

In [10]:
zip_mean_lat_long.head()

,geolocation_lat,geolocation_lng
geolocation_zip_code_prefix,,
1001,-23.550190,-46.634024
1002,-23.548146,-46.634979
1003,-23.548994,-46.635731
1004,-23.549799,-46.634757
1005,-23.549456,-46.636733


In [11]:
# merging using left join with geolocation table to extract state,zipcode,lat and long
state_zip_lat_lng = pd.merge(zip_mean_lat_long,geo_loc,on='geolocation_zip_code_prefix',how='left')[['geolocation_state','geolocation_zip_code_prefix','geolocation_lat_x','geolocation_lng_x']]

In [12]:
state_zip_lat_lng

,geolocation_state,geolocation_zip_code_prefix,geolocation_lat_x,geolocation_lng_x
0,SP,1001,-23.550190,-46.634024
1,SP,1001,-23.550190,-46.634024
2,SP,1001,-23.550190,-46.634024
3,SP,1001,-23.550190,-46.634024
4,SP,1001,-23.550190,-46.634024
...,...,...,...,...
1000158,RS,99980,-28.389129,-51.843836
1000159,RS,99980,-28.389129,-51.843836
1000160,RS,99980,-28.389129,-51.843836
1000161,RS,99990,-28.329595,-51.769362


In [13]:
state_zip_lat_lng.duplicated().value_counts()

True     981140
False     19023
dtype: int64

In [14]:
state_zip_lat_lng['geolocation_zip_code_prefix'].nunique()

19015

In [15]:
# removing duplicates and keeping only one lat and long for one zip code
state_zip_lat_lng = state_zip_lat_lng.drop_duplicates()
state_zip_lat_lng.to_csv('./state_zip_lat_lng.csv',index=False) # saving it to csv file with no index

In [16]:
sellers = pd.read_csv('./sellers.csv')
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [17]:
customers = pd.read_csv('./customers.csv')

In [18]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [19]:
# meging sellers with state_zip_lat_lng to generate a composite key based on seller id and zip code 
sellers_juction_table = state_zip_lat_lng.merge(sellers,left_on='geolocation_zip_code_prefix',right_on='seller_zip_code_prefix',how='inner')[['seller_id','geolocation_zip_code_prefix']]

In [20]:
sellers_juction_table

,seller_id,geolocation_zip_code_prefix
0,8602a61d680a10a82cceeeda0d99ea3d,1001
1,e0487761face83d64fcada2408959a36,1021
2,dd55f1bb788714a40e7954c3be6df745,1021
3,09bad886111255c5b5030314fc7f1a4a,1022
4,f049a72cf58fd31b11f8919cade515e7,1023
...,...,...
3084,447d377bdb757058acb569025ee18a93,99500
3085,b1a81260566c1bac3114a6d124413f27,99500
3086,4fae87d32467e18eb46e4a76a0a0b9ce,99670
3087,968ee78631915a63fef426d6733d7422,99700


In [21]:
sellers_juction_table.to_csv('./seller_junction_table.csv',index=False)

In [22]:
# meging customer with state_zip_lat_lng to generate a composite key based on customer id and zip code 
customers_juction_table = state_zip_lat_lng.merge(customers,left_on='geolocation_zip_code_prefix',right_on='customer_zip_code_prefix',how='inner')[['customer_id','geolocation_zip_code_prefix']]

In [23]:
customers_juction_table

,customer_id,geolocation_zip_code_prefix
0,7ae2a9337aa4bc799723511faa1d6830,1003
1,a09edf8c1e842e94805a206b3d73eed5,1004
2,ee9b73e88afb4904ee2322cfc89cf638,1004
3,5a8b64ee6ccdae09ea823e6aa00e9517,1005
4,6ec2b4682814cfdac8d92bad42b3ddab,1005
...,...,...
99232,235702411e5214f0e4a0761bf5ce9e99,99965
99233,3ab8bc00f8740d54afc4c771fb6c7f69,99970
99234,964b34423c822e9bd9831635ccc83db2,99980
99235,657ba09c6edfbbc09f6054f541ec1f90,99980


In [24]:
customers_juction_table.to_csv('./customer_junction_table.csv',index=False)

In [56]:
orders=pd.read_csv('./orders.csv')
orders.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

In [57]:
customers.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')

In [58]:
pd.merge(orders,customers,how='inner',on='customer_id')[['customer_id','customer_state','order_id']]

,customer_id,customer_state,order_id
0,9ef432eb6251297304e76186b10a928d,SP,e481f51cbdc54678b7cc49136f2d6af7
1,b0830fb4747a6c6d20dea0b8c802d7ef,BA,53cdb2fc8bc7dce0b6741e2150273451
2,41ce2a54c0b03bf3443c3d931a367089,GO,47770eb9100c2d0c44946d9cf07ec65d
3,f88197465ea7920adcdbec7375364d82,RN,949d5b44dbf5de918fe9c16f97b45f8a
4,8ab97904e6daea8866dbdbc4fb7aad2c,SP,ad21c59c0840e6cb83a9ceb5573f8159
...,...,...,...
99436,39bd1228ee8140590ac3aca26f2dfe00,SP,9c5dedf39a927c1b2549525ed64a053c
99437,1fca14ff2861355f6e5f14306ff977a7,SP,63943bddc261676b46f01ca7ac2f7bd8
99438,1aa71eb042121263aafbe80c1b562c9c,BA,83c1379a015df1e13d02aae0204711ab
99439,b331b74b18dc79bcdf6532d51e1637c1,RJ,11c177c8e97725db2631073c19f07b62


In [77]:
orders_per_cust_state =orders_cust_state.groupby('customer_state').agg({"order_id":"count","customer_id":"count"})

In [62]:
order_items =pd.read_csv("./order_items.csv")
# order_items.columns
# sellers.columns

In [63]:
order_items

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [64]:
sellers.columns

Index(['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'], dtype='object')

In [71]:
order_items_seller =pd.merge(order_items,sellers,how='left',on='seller_id',suffixes=('_x','_y'))

In [75]:
orders_per_seller_state = order_items_seller.groupby('seller_state').agg({"order_id":"count"})

In [76]:
orders_per_seller_state

,order_id
seller_state,
AC,1
AM,3
BA,643
CE,94
DF,899
ES,372
GO,520
MA,405
MG,8827


In [80]:
orders_per_seller_state.to_csv("./orders_per_seller_state.csv")

In [81]:
orders_per_cust_state.to_csv("./orders_per_cust_state.csv")